# 1. Gerar Nova Planilha de Mensalidades

Este notebook é responsável pela etapa principal do processamento. Ele realiza a leitura dos dados brutos e atualiza os valores das mensalidades com base na planilha fornecida pela operadora.

**Entradas necessárias:**
- Planilha base (Sheet 'AGO 2025')
- Planilha de mensalidades (Relatório da operadora)

**Saída:**
- Arquivo `dados_atualizados.xlsx`

In [ ]:
import pandas as pd

### 1.1 Carregamento dos Dados
Carrega as planilhas do Excel. 
**Nota:** Certifique-se de preencher os caminhos dos arquivos nas funções `read_excel` abaixo antes de executar.

In [ ]:
# Carrega os dados da planilha principal e da planilha de mensalidades
dados = pd.read_excel('../dados.xlsx', skiprows=1, sheet_name='OUT 2025')
mensalidades = pd.read_excel('../mensalidades.xlsx', index_col=0)

# Remove as últimas 35 linhas (rodapés ou dados indesejados)
dados = dados.iloc[:-35]

### 1.2 Visualização Inicial
Exibe as primeiras linhas para garantir que os dados foram carregados corretamente.

In [ ]:
dados.head()

In [ ]:
mensalidades.head()

### 1.3 Atualização dos Valores
Itera sobre a planilha de mensalidades e atualiza a coluna `MENSALIDADE` na planilha principal (`dados`) correspondente ao nome encontrado.

**Melhoria:** Realiza limpeza de espaços nos nomes para garantir melhor correspondência.

In [ ]:
# Limpeza de espaços em branco para garantir correspondência exata
dados['NOME'] = dados['NOME'].astype(str).str.strip()
mensalidades['Nome'] = mensalidades['Nome'].astype(str).str.strip()

# Opcional: Converter para maiúsculas para evitar problemas de case-sensitive
# dados['NOME'] = dados['NOME'].str.upper()
# mensalidades['Nome'] = mensalidades['Nome'].str.upper()

contador_atualizados = 0

for index, row in mensalidades.iterrows():
    nome_mensalidade = row['Nome']
    valor_mensalidade = row['Valor']

    # Verifica se o nome existe na planilha dados
    if nome_mensalidade in dados['NOME'].values:
        # Encontra a linha correspondente em 'dados' e ATUALIZA a 'MENSALIDADE'
        # A atribuição direta funciona tanto para células vazias (NaN) quanto preenchidas
        dados.loc[dados['NOME'] == nome_mensalidade, 'MENSALIDADE'] = valor_mensalidade
        contador_atualizados += 1

print(f"Total de registros atualizados/preenchidos: {contador_atualizados}")
display(dados.head())

### 1.4 Verificação Pós-Atualização
Exibe uma amostra maior dos dados para conferência visual.

In [ ]:
dados.head(10)

### 1.5 Exportação
Salva o resultado processado em um novo arquivo Excel.

In [ ]:
dados.to_excel('dados_atualizados.xlsx', index=False)